<a href="https://colab.research.google.com/github/outrocisco/Explora-/blob/main/Explora%C3%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# @title 1. Instalação e Configuração Inicial (Corrigido Nome do Secret)
# @markdown Instala as bibliotecas e configura a API Key do Gemini.
!pip install -q google-genai google-adk ipywidgets

import os
from google.colab import userdata # Para Colab Secrets
from google import genai # << Nova forma de importar e acessar o cliente >>

import json
import re
from datetime import datetime, timedelta
import pytz
import urllib.parse

# Para exibir HTML na saída
from IPython.display import display, HTML, Markdown


# Importa a ferramenta de busca do ADK (necessária para a sintaxe de tools na chamada generate_content)
from google.adk.tools import google_search


# --- Configuração da API Key do Gemini ---
try:
    # >> CORRIGIDO O NOME DO SECRET PARA 'GOOGLE_API_KEY_3' <<
    API_KEY_SECRET_NAME = 'GOOGLE_API_KEY_3' # << Use o nome exato do seu Secret no Colab >>
    API_KEY_VALUE = userdata.get(API_KEY_SECRET_NAME)

    if not API_KEY_VALUE:
        print(f"⚠️ API Key do Gemini não encontrada nos Colab Secrets (nome: {API_KEY_SECRET_NAME}).")
        print("Por favor, adicione seu Secret com este nome (🔑 ícone na barra lateral esquerda).")
        pass # Continuar, mas a configuração do cliente abaixo provavelmente falhará
    else:
        os.environ["GOOGLE_API_KEY"] = API_KEY_VALUE # Define a variável de ambiente
        print(f"✅ API Key do Gemini configurada usando o Secret '{API_KEY_SECRET_NAME}'.")


except Exception as e:
    print(f"❌ Erro inesperado ao configurar API Key do Gemini: {e}")
    # A chave não foi configurada, a inicialização do cliente abaixo provavelmente falhará


# --- Configura o cliente e o modelo Gemini ---
client = None
MODEL_ID = "gemini-2.0-flash"

try:
    print("🔄 Tentando configurar o cliente e carregar o ID do modelo...")
    # A inicialização de Client() tenta ler a chave da variável de ambiente GOOGLE_API_KEY
    # Se a chave não foi configurada corretamente acima, este passo vai falhar
    client = genai.Client()

    if client and MODEL_ID:
         print(f"✅ Cliente Gemini configurado. Modelo ID definido como '{MODEL_ID}'.")
         print("ℹ️ A capacidade de busca (google_search) será configurada diretamente na chamada de geração na Célula 3.")
    else:
         print("❌ Cliente Gemini ou Modelo ID não configurado.")
         print("Verifique a Célula 1 e sua API Key.")


except Exception as e:
    print(f"❌ Erro ao configurar o cliente Gemini: {e}")
    client = None
    print("Sugestões de IA (especialmente as baseadas em busca real) não funcionarão.")
    print("\n>> CAUSA PROVÁVEL DO ERRO ATUAL <<")
    print(f">> O cliente Gemini falhou ao inicializar, provavelmente porque a API Key (Secret '{API_KEY_SECRET_NAME}') não foi carregada corretamente ou não é válida/não tem permissão.")
    print(">> Verifique novamente se o Secret no Colab tem o nome correto ('GOOGLE_API_KEY_3'), se a chave é válida e se tem permissão para usar o modelo COM FERRAMENTAS no Google Cloud Console/AI Studio.")

✅ API Key do Gemini configurada usando o Secret 'GOOGLE_API_KEY_3'.
🔄 Tentando configurar o cliente e carregar o ID do modelo...
✅ Cliente Gemini configurado. Modelo ID definido como 'gemini-2.0-flash'.
ℹ️ A capacidade de busca (google_search) será configurada diretamente na chamada de geração na Célula 3.


In [42]:
# @title 2. Perfil do Usuário (Inputs Formatados Melhor - v3)
# @markdown Define, carrega ou edita as informações do usuário e o progresso simples.
# >> CONFIGURAÇÃO DE PERSISTÊNCIA (OPCIONAL) <<
# Para que seu perfil seja salvo e carregado automaticamente entre sessões,
# monte seu Google Drive no Colab e ajuste o caminho PROFILE_PATH.
# Exemplo (descomente as 2 linhas abaixo):
# from google.colab import drive
# drive.mount('/content/drive')
PROFILE_PATH = 'explorae_profile.json' # Ou '/content/drive/MyDrive/explorae_profile.json'

user_profile = {}
# Tentar carregar perfil existente
if os.path.exists(PROFILE_PATH):
    try:
        with open(PROFILE_PATH, 'r') as f:
            user_profile = json.load(f)
        # "Bem-vindo(a) de volta" via print, não centralizado
        print(f"👋 Bem-vindo(a) de volta, {user_profile.get('nome', 'Explorador(a)')}!")
        print("Seu perfil foi carregado.")
        # Lidar com campos renomeados/removidos de versões anteriores (mantido para compatibilidade)
        if 'interesses' in user_profile:
            del user_profile['interesses']
        if 'vibe' in user_profile:
            del user_profile['vibe']
        if 'horarios_livres' in user_profile and 'horarios_que_costuma_ter_libre' not in user_profile:
             user_profile['horarios_que_costuma_ter_libre'] = user_profile.pop('horarios_livres')
        elif 'horarios_que_costuma_ter_libre' not in user_profile:
             user_profile['horarios_que_costuma_ter_libre'] = "" # Garante que o campo existe
        if 'localizacao' in user_profile and 'cidade_e_bairro' not in user_profile:
             user_profile['cidade_e_bairro'] = user_profile.pop('localizacao')
        elif 'cidade_e_bairro' not in user_profile:
             user_profile['cidade_e_bairro'] = "" # Garante que o campo existe

        # Re-adicionar ou garantir campos de Gamificação/Histórico simplificados
        if 'rolês_explorados' not in user_profile:
            # Tratar caso venha de um perfil antigo com 'agendamentos_feitos'
            if 'agendamentos_feitos' in user_profile:
                user_profile['rolês_explorados'] = user_profile.pop('agendamentos_feitos')
            else:
                 user_profile['rolês_explorados'] = 0 # Inicializa se não existir
        if 'historico_rolês' not in user_profile:
             user_profile['historico_rolês'] = [] # Inicializa se não existir


        # Garantir novos campos básicos se o perfil antigo não os tinha
        if 'tipo_role' not in user_profile:
             user_profile['tipo_role'] = ""
        if 'complemento_role' not in user_profile:
             user_profile['complemento_role'] = ""


    except Exception as e:
        print(f"❌ Erro ao carregar perfil: {e}")
        user_profile = {} # Resetar se houver erro no carregamento

# Se o perfil não foi carregado ou está incompleto, solicitar informações
# Considera o perfil incompleto se campos chave como nome, tipo_role, horarios e localizacao estiverem vazios/Nulos
if not user_profile or not all(user_profile.get(k) for k in ['nome', 'tipo_role', 'horarios_que_costuma_ter_libre', 'cidade_e_bairro']): # 'complemento_role' pode ser vazio
    # Mensagem inicial estilizada para configurar perfil
    display(HTML(formatar_config_perfil_html()))

    # --- INPUTS COM QUEBRA DE LINHA NO FINAL (\n) ---
    user_profile['nome'] = input("Qual é o seu nome?\n").strip() # \n no final

    # Combinado o print da pergunta com o input dos exemplos e quebra de linha no final
    user_profile['tipo_role'] = input("Que tipo de rolê você está procurando agora?\nExemplos: um date, rolê com amigos, aniversário, caminhada, dançar, karaokê, só beber, algo diferente: \n").strip() # \n no final

    # Combinado o print da pergunta com o input dos exemplos e quebra de linha no final
    user_profile['complemento_role'] = input("Alguma informação adicional para me ajudar a refinar a busca?\nExemplos: quero algo tranquilo, animado, com música ao vivo, perto do trabalho, que aceite pets que tenha comida vegana: \n").strip() # \n no final

    user_profile['horarios_que_costuma_ter_libre'] = input("Quais horários você costuma ter livre (ex: fins de semana, noites de terças)?\n").strip() # \n no final
    # --- PERGUNTA DE LOCALIZAÇÃO ATUALIZADA E COM QUEBRA DE LINHA NO FINAL ---
    user_profile['cidade_e_bairro'] = input("Qual a localização ideal? Diga a Cidade e o Bairro para a aventura!\n").strip() # \n no final


    # Inicializa campos de Gamificação/Histórico simplificados se for um novo perfil
    if 'rolês_explorados' not in user_profile: user_profile['rolês_explorados'] = 0
    if 'historico_rolês' not in user_profile: user_profile['historico_rolês'] = []


    # Salvar novo perfil (Opcional)
    try:
        with open(PROFILE_PATH, 'w') as f:
            json.dump(user_profile, f, indent=4)
        print(f"\n✅ Perfil de {user_profile['nome']} criado e salvo em {PROFILE_PATH}.")
    except Exception as e:
         print(f"\n⚠️ Não foi possível salvar o perfil automaticamente: {e}")
         print("Você pode salvar manualmente após a execução (Célula 5).")

# Exibe resumo do perfil, incluindo gamificação/histórico
print("\n--- Seu Perfil de Explorador(a) ---")
print(f"Nome: {user_profile.get('nome', 'N/A')}")
print(f"Tipo de Rolê Buscado: {user_profile.get('tipo_role', 'N/A')}")
print(f"Informação Complementar: {user_profile.get('complemento_role', 'N/A')}")
print(f"Horários Livres: {user_profile.get('horarios_que_costuma_ter_libre', 'N/A')}")
print(f"Cidade e Bairro: {user_profile.get('cidade_e_bairro', 'N/A')}")
print(f"Explorações Concluídas: {user_profile.get('rolês_explorados', 0)}")
# Exibe histórico de forma simples, apenas os nomes
historico_nomes = [item.get('NOME', 'Lugar Desconhecido') for item in user_profile.get('historico_rolês', [])][-5:] # Últimos 5
print(f"Histórico Recente: {', '.join(historico_nomes) if historico_nomes else 'Vazio'}")
print("-----------------------------------")


def editar_perfil(profile):
    """Permite ao usuário editar informações do perfil."""
    print("\n--- Editar Seu Perfil ---")
    print("Deixe em branco para manter o valor atual.")

    # Usar .get() para exibir valores atuais sem erro se o campo não existir (útil para perfis antigos)
    current_tipo_role = profile.get('tipo_role', 'Não definido')
    # --- INPUTS DE EDIÇÃO COM QUEBRA DE LINHA NO FINAL (\n) ---
    novo_tipo_role = input(f"\nTipo de rolê atual ({current_tipo_role}): Novo tipo de rolê: \n").strip() # \n no final
    if novo_tipo_role:
        profile['tipo_role'] = novo_tipo_role
        print("✅ Tipo de rolê atualizado.")

    current_complemento = profile.get('complemento_role', 'Não definido')
    novo_complemento = input(f"\nInfo complementar atual ({current_complemento}): Nova info complementar: \n").strip() # \n no final
    if novo_complemento:
        profile['complemento_role'] = novo_complemento
        print("✅ Informação complementar atualizada.")

    current_horarios = profile.get('horarios_que_costuma_ter_libre', 'Não definido')
    novo_horarios = input(f"\nHorários livres atuais ({current_horarios}): Novos horários livres: \n").strip() # \n no final
    if novo_horarios:
        profile['horarios_que_costuma_ter_libre'] = novo_horarios
        print("✅ Horários livres atualizados.")

    current_localizacao = profile.get('cidade_e_bairro', 'Não definido')
    novo_localizacao = input(f"\nCidade e bairro atuais ({current_localizacao}): Nova cidade e bairro: \n").strip() # \n no final
    if novo_localizacao:
        profile['cidade_e_bairro'] = novo_localizacao
        print("✅ Cidade e bairro atualizada.")

    # Removido: Opção de editar contador ou histórico diretamente via input aqui

    print("--- Edição Concluída ---")
    # A Célula 4 cuidará de regenerar sugestões e salvar.

👋 Bem-vindo(a) de volta, Francisco Lucas!
Seu perfil foi carregado.

--- Seu Perfil de Explorador(a) ---
Nome: Francisco Lucas
Tipo de Rolê Buscado: Quero fazer um aniversário
Informação Complementar: comida vegana e bebidas
Horários Livres: Final de semana
Cidade e Bairro: São Paulo
Explorações Concluídas: 0
Histórico Recente: Vazio
-----------------------------------


In [46]:
# @title 3. Funções Auxiliares e Estilização (Workaround para Erro de Ferramenta)
# @markdown Define as funções Python que o agente usará, incluindo formatação HTML.
# @markdown IMPORTANTE: A ferramenta de busca foi temporariamente desabilitada para evitar um erro de validação da API.

def gerar_sugestoes_texto(perfil, tentar_novas=False):
    """Gera sugestões de lugares/eventos usando o modelo Gemini."""
    # Verifica se o cliente e o MODEL_ID foram configurados corretamente na Célula 1
    if client is None or MODEL_ID is None:
        print("❌ Cliente Gemini ou Modelo ID não configurado na Célula 1. Não é possível gerar sugestões.")
        return ""

    # Re-adicionado: Adicionar histórico ao prompt para tentar influenciar (ainda útil para influenciar o modelo mesmo sem busca real)
    historico_nomes = [item.get('NOME', '') for item in perfil.get('historico_rolês', [])][-7:] # Últimos 7 no histórico
    historico_str = f"Lugares que o usuário já explorou recentemente (EVITE SUGERIR ESTES OU LUGARES MUITO SIMILARES NOVAMENTE): {', '.join(historico_nomes)}" if historico_nomes else "O usuário não tem histórico de explorações recentes."


    prompt = f"""
    Você é o agente Exploraê, um assistente amigável e criativo focado em encontrar lugares e eventos (baseado no seu conhecimento) para {perfil.get('nome', 'o usuário')} explorar em {perfil.get('cidade_e_bairro', 'sua localização')}.
    O usuário está procurando por: **{perfil.get('tipo_role', 'um rolê geral')}**.
    Informação adicional: **{perfil.get('complemento_role', 'nenhuma informação complementar fornecida')}**.
    Horários geralmente livres: {perfil.get('horarios_que_costuma_ter_libre', 'não especificados')}.
    {historico_str}

    Tarefa: Sugira 3 opções de lugares ou eventos diferentes e interessantes que combinem com o **tipo de rolê**, a **informação complementar** e a **localização** do usuário, usando apenas o seu conhecimento interno.
    Procure por lugares diferentes, únicos ou que ofereçam experiências fora do comum, mas que ainda se encaixem no tipo de rolê buscado.
    Se {tentar_novas} for True, tente encontrar opções significativamente diferentes das sugestões anteriores (pense em tipos de lugar, bairros, atividades, e evite os nomes do histórico).

    Formato de saída EXATO desejado para cada opção (separe por "---"). Inclua apenas informações que você pode gerar com base no seu conhecimento:
    NOME: [Nome de Local/Evento Sugerido (pode ser genérico se não houver conhecimento específico)]
    DESCRICAO: [Breve Descrição (1-2 frases)]
    TIPO: [Tipo do Local (Café, Bar, Parque, Clube de Dança, Centro Cultural, etc.)]
    LOCALIZACAO: [Localização Aproximada (Cidade/Bairro, ou "Na sua região")]
    MOTIVO_RECOMENDACAO: [Por que este lugar é perfeito para o tipo de rolê e informação complementar do usuário? (1 frase)]
    ---

    Now, generate the 3 different suggestions for {perfil.get('nome', 'the user')} in {perfil.get('cidade_e_bairro', 'their location')}:
    """
    try:
        # >> USANDO O CLIENTE E MODEL_ID CONFIGURADOS NA CÉLULA 1 <<
        # >> WORKAROUND: Removida a configuração da ferramenta de busca para evitar erro de validação <<
        response = client.models.generate_content(
             model=MODEL_ID,
             contents=prompt,
             # config={"tools": [{"Google Search": {}}]} # LINHA COMENTADA/REMOVIDA
        )

        if not response or not response.text:
             print("❌ A IA não retornou um texto de sugestão válido.")
             # Opcional: Inspecionar response.prompt_feedback para ver motivos de segurança, etc.
             return ""

        # Os metadados de busca não estarão disponíveis com o workaround
        # try:
        #     if response.candidates and response.candidates[0].grounding_metadata and response.candidates[0].grounding_metadata.web_search_queries:
        #         print(f"🔍 Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
        # except Exception as e:
        #     pass


        return response.text

    except Exception as e:
        print(f"❌ Erro ao gerar sugestões (API Gemini): {e}")
        print("Verifique se o cliente e o Modelo ID estão configurados na Célula 1 e se há problemas de API durante a geração.")
        # A mensagem de causa provável relacionada a ferramentas não é mais relevante com o workaround
        return ""

def parse_sugestoes(texto_raw):
    """Parseia o texto da resposta da IA em uma lista de dicionários."""
    sugestoes = []
    opcoes_raw = texto_raw.strip().split('---')
    for opcao_raw in opcoes_raw:
        if not opcao_raw.strip():
            continue
        sugestao = {}
        lines = opcao_raw.strip().splitlines()
        for line in lines:
            if ':' in line:
                try:
                    key, value = line.split(':', 1)
                    sugestao[key.strip()] = value.strip()
                except ValueError:
                    pass # Ignore malformed line
        if sugestao:
            # Ensure expected keys exist, even if with default 'N/A'
            parsed_sugestao = {
                'NOME': sugestao.get('NOME', 'Nome não disponível').strip(),
                'DESCRICAO': sugestao.get('DESCRICAO', 'N/A').strip(),
                'TIPO': sugestao.get('TIPO', 'N/A').strip(),
                'LOCALIZACAO': sugestao.get('LOCALIZACAO', 'N/A').strip(),
                'MOTIVO_RECOMENDACAO': sugestao.get('MOTIVO_RECOMENDACAO', 'N/A').strip() # New field
            }
            # Check if at least NOME has a real value (avoid empty cards or "Nome não disponível")
            if parsed_sugestao.get('NOME', '') != '' and parsed_sugestao.get('NOME') != 'Nome não disponível':
                 sugestoes.append(parsed_sugestao)
    return sugestoes

def formatar_sugestoes_html(sugestoes):
    """Formata a lista de sugestões como HTML estilizado (cards) com links do Maps."""
    # Estilos CSS COMPLETO e ATUALIZADO com cores de maior contraste para detalhes
    css_style = """
    <style>
      .sugestoes-container {
          display: flex;
          flex-wrap: wrap;
          gap: 20px;
          margin-bottom: 20px;
      }
      .sugestao-card {
        border: 1px solid #b22222; /* Vermelho tijolo borda */
        border-radius: 15px; /* Bordas mais arredondadas */
        padding: 20px;
        margin-bottom: 15px;
        background-color: #ffebeb; /* Vermelho bem claro fundo */
        font-family: 'Arial', sans-serif; /* Fonte simples */
        max-width: 350px; /* Limita a largura para parecer um card */
        box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.1); /* Sombra suave */
        color: #333; /* Cor de texto padrão */
        word-wrap: break-word; /* Garante quebra de texto longa */
        flex-grow: 1; /* Permite que os cards cresçam para preencher o espaço */
        display: flex; /* Usa flexbox para alinhar conteúdo interno */
        flex-direction: column; /* Empilha conteúdo interno verticalmente */
        justify-content: space-between; /* Espalha conteúdo interno verticalmente */
      }
      /* Estilo para o card de detalhes da escolha - CORES AJUSTADAS PARA MAIOR CONTRASTE */
      .detalhe-card {
        border: 2px solid #dc143c; /* Carmesim borda mais forte */
        border-radius: 15px;
        padding: 25px; /* Um pouco mais de padding */
        margin: 20px auto; /* Centraliza e dá margem */
        background-color: #fff0f3; /* Rosa bem claro fundo */
        font-family: 'Arial', sans-serif;
        max-width: 550px; /* Pode ser um pouco mais largo */
        box-shadow: 4px 4px 10px rgba(0, 0, 0, 0.15); /* Sombra mais forte */
        color: #660000; /* COR VERMELHA ESCURA PARA TEXTO GERAL */
        word-wrap: break-word; /* Garante quebra de texto longa */
      }
      .detalhe-card h3 {
        margin-top: 0;
        color: #8b0000; /* Vermelho escuro para o título */
        font-size: 1.4em; /* Título um pouco maior */
        border-bottom: 2px solid #dc143c; /* Linha separadora mais forte */
        padding-bottom: 10px;
        margin-bottom: 15px;
        line-height: 1.3;
      }
       .detalhe-card p {
        margin-bottom: 10px; /* Mais espaço entre parágrafos */
        line-height: 1.5;
        color: #660000; /* COR VERMELHA ESCURA PARA PARÁGRAFOS */
        text-align: justify; /* Justifica o texto */
      }
       .detalhe-card p:last-child {
        margin-bottom: 0;
      }
      .sugestao-card h3 {
        /* ESTE É O ESTILO DO TÍTULO DO CARD DE SUGESTÃO - COR AJUSTADA */
        color: #660000; /* <<< AJUSTADO: MESMO VERMELHO ESCURO DO TÍTULO DO DETALHE >>> */
        line-height: 1.3; /* Espaçamento de linha para títulos longos */
        margin-top: 0; /* Garante que não tem margem superior extra */
        border-bottom: 1px dashed #b22222; /* Linha separadora */
        padding-bottom: 10px; /* Espaço abaixo da linha */
      }
      .sugestao-card p {
         /* Estilos compartilhados para parágrafos */
         text-align: justify; /* Justifica o texto para visual melhor */
         flex-grow: 1; /* Permite que a descrição ocupe espaço extra */
         color: #333; /* Cor de texto padrão para o card de sugestão */
         margin-bottom: 8px; /* Margem padrão para parágrafos do card */
         line-height: 1.4; /* Espaçamento de linha padrão para o card */
      }
       .sugestao-card .label, .detalhe-card .label {
        font-weight: bold;
        color: #800000; /* VERMELHO ESCURO MAIS FORTE PARA RÓTULOS */
      }
       .sugestao-card .option-number {
           display: inline-block;
           background-color: #dc143c; /* Carmesim */
           border-radius: 50%; /* Redondo */
           width: 30px;
           height: 30px;
           text-align: center;
           line-height: 30px;
           margin-right: 10px;
           font-weight: bold;
           color: white; /* Cor do texto do número */
           flex-shrink: 0; /* Não encolher no flexbox */
       }
       .card-header {
           display: flex;
           align-items: center;
           margin-bottom: 10px;
       }
       .map-link, .calendar-link { /* Estilos compartilhados para links */
           display: block; /* Link em sua própria linha */
           margin-top: 10px; /* Espaço acima */
           padding: 8px 0; /* Padding para área clicável */
           color: #dc143c; /* Carmesim para o link */
           text-decoration: none; /* Sem sublinhado */
           font-weight: bold;
           border-top: 1px solid #f0d0d0; /* Linha separadora acima dos links */
       }
       .map-link:hover, .calendar-link:hover {
           text_decoration: underline; /* Sublinhado ao passar o mouse */
       }
        .calendar-link {
             border-top: none; /* Remove a borda dupla entre os links */
        }


       /* Estilo para as instruções de input - Ajustando Largura */
       .input-instructions {
           margin: 20px auto; /* Centraliza e dá margem */
           padding: 15px;
           border: 1px dashed #b22222; /* Borda vermelha */
           border-radius: 8px;
           background-color: #ffebeb; /* Fundo vermelho claro */
           font-family: Arial, sans-serif;
           color: #333; /* Texto escuro para contraste */
           max-width: 550px; /* <<< Ajuste de largura aqui >>> */
           word-wrap: break-word; /* Garante quebra de texto */
       }
       .input-instructions p {
           font-weight: bold;
           color: #8b0000; /* Vermelho escuro */
           margin-bottom: 10px;
       }
       .input-instructions ul {
           list-style-type: disc;
           margin-left: 20px;
           padding-left: 0; /* Remove padding padrão da ul */
       }
        .input-instructions li {
           margin-bottom: 5px;
           color: #555; /* Texto mais suave para os itens da lista */
       }
        /* Estilo para a introdução */
        .introduction-box {
            margin: 20px auto; /* Centraliza */
            padding: 25px;
            border: 2px solid #a52a2a; /* Borda marrom */
            border-radius: 10px;
            background-color: #fff8f8; /* Fundo quase branco/muito claro */
            font-family: 'Arial', sans-serif;
            max-width: 600px; /* Largura para a caixa de introdução */
            box-shadow: 2px 2px 6px rgba(0,0,0,0.08);
            text-align: center; /* Centraliza texto */
            word-wrap: break-word; /* Garante quebra de texto */
        }
        .introduction-box h1 {
            color: #8b0000; /* Vermelho escuro */
            font-size: 1.8em;
            margin-top: 0;
            margin-bottom: 15px;
        }
         .introduction-box p {
            color: #555;
            font-size: 1.1em;
            line-height: 1.6;
            margin-bottom: 15px;
         }
         .introduction-box .highlight {
             color: #dc143c; /* Carmesim para destacar */
             font-weight: bold;
         }
         /* Estilo para a mensagem final */
         .final-message {
            margin: 30px auto;
            padding: 25px;
            border: 2px dashed #8b0000;
            border-radius: 10px;
            background-color: #ffebeb;
            font-family: 'Arial', sans-serif;
            max-width: 550px; /* Ajuste de largura */
            text-align: center;
            word-wrap: break-word;
         }
         .final-message h2 {
            color: #dc143c;
            margin-top: 0;
            margin-bottom: 15px;
         }
         .final-message p {
            color: #333; /* Texto mais escuro */
            margin-bottom: 10px;
            line-height: 1.5;
         }
          .final-message .title-highlight {
              color: #800000; /* Cor mais escura para destaque do título */
              font-weight: bold;
          }
        /* Estilo para a mensagem de configuração de perfil inicial */
         .config-message {
            margin: 20px auto;
            padding: 20px;
            border: 1px solid #a52a2a;
            border-radius: 8px;
            background-color: #fff8f8;
            font-family: Arial, sans-serif;
            max-width: 550px; /* Ajuste de largura */
            word-wrap: break-word;
         }
         .config-message h2 {
             color: #8b0000;
             margin-top: 0;
             margin-bottom: 10px;
         }
         .config-message p {
             color: #555;
             line-height: 1.5;
         }

    </style>
    """ # FIM DA STRING css_style

    html_content = css_style # Adiciona os estilos
    html_content += "<h2>✨ Suas Aventuras Sugeridas ✨</h2>"
    html_content += "<div class='sugestoes-container'>" # Container para flexbox

    if not sugestoes:
        html_content += "<p>Não foi possível gerar sugestões no momento.</p>"
        html_content += "</div>"
        # Adiciona as instruções de input mesmo que não haja sugestões, para que o usuário saiba o que fazer
        html_content += formatar_instrucoes_input_html()
        return html_content

    for i, sugestao in enumerate(sugestoes):
        num = i + 1 # Números das opções (1, 2, 3)

        # Constrói a URL de busca do Google Maps (usado nos cards de sugestão)
        query = f"{sugestao.get('NOME', '')} {sugestao.get('LOCALIZACAO', '')}".strip()
        encoded_query = urllib.parse.quote_plus(query)
        map_search_url = f"https://www.google.com/maps/search/?api=1&query={encoded_query}"


        html_content += f"""
        <div class="sugestao-card">
          <div class="card-header">
              <span class="option-number">{num}</span>
              <h3>{sugestao.get('NOME', 'Nome não disponível')}</h3>
          </div>
          <p><span class="label">Descrição:</span> {sugestao.get('DESCRICAO', 'N/A')}</p>
          <p><span class="label">Tipo:</span> {sugestao.get('TIPO', 'N/A')}</p>
          <p><span class="label">Localizacao:</span> {sugestao.get('LOCALIZACAO', 'N/A')}</p>
          <p><span class="label">Motivo da Recomendacao:</span> {sugestao.get('MOTIVO_RECOMENDACAO', 'N/A')}</p>
          <a href="{map_search_url}" target="_blank" class="map-link">📍 Ver no Google Maps</a>
        </div>
        """
    html_content += "</div>" # Fecha o container flexbox

    # Adiciona as instruções para interação em um bloco estilizado
    html_content += formatar_instrucoes_input_html()

    return html_content


def formatar_instrucoes_input_html():
     """Gera o HTML para as instruções de input, estilizado com largura limitada."""
     return """
     <div class="input-instructions">
         <p>👀 Dê uma olhada nas opções acima! Digite:</p>
         <ul>
             <li><strong>1, 2 ou 3:</strong> para saber mais sobre a opção escolhida.</li>
             <li><strong>'n':</strong> para ver novas sugestões.</li>
             <li><strong>'e':</strong> para editar seu perfil e preferências.</li>
         </ul>
     </div>
     """

def formatar_detalhes_html(sugestao):
    """Gera o HTML estilizado para exibir os detalhes de uma sugestão escolhida."""
    # Constrói a URL de busca do Google Maps
    query = f"{sugestao.get('NOME', '')} {sugestao.get('LOCALIZACAO', '')}".strip()
    encoded_query = urllib.parse.quote_plus(query)
    map_search_url = f"https://www.google.com/maps/search/?api=1&query={encoded_query}"

    # Constrói a URL para criar evento no Google Calendar (atalho)
    # Adiciona título e localização. Data/Hora precisam ser preenchidas pelo usuário no Calendar.
    calendar_title = f"Exploraê: {sugestao.get('NOME', 'Rolê Sugerido')}"
    calendar_location = sugestao.get('LOCALIZACAO', 'Local não especificado')
    encoded_cal_title = urllib.parse.quote_plus(calendar_title)
    encoded_cal_location = urllib.parse.quote_plus(calendar_location)
    # Link básico para criar evento: https://calendar.google.com/calendar/render?action=TEMPLATE&text=TITULO&location=LOCAL
    calendar_shortcut_url = f"https://calendar.google.com/calendar/render?action=TEMPLATE&text={encoded_cal_title}&location={encoded_cal_location}"


    html_content = f"""
    <div class="detalhe-card">
      <h3>🥳 Você escolheu: {sugestao.get('NOME', 'Opção desconhecida')}</h3>
      <p><span class="label">Descrição:</span> {sugestao.get('DESCRICAO', 'N/A')}</p>
      <p><span class="label">Tipo:</span> {sugestao.get('TIPO', 'N/A')}</p>
      <p><span class="label">Localização:</span> {sugestao.get('LOCALIZACAO', 'N/A')}</p>
      <p><span class="label">Motivo da Recomendação:</span> {sugestao.get('MOTIVO_RECOMENDACAO', 'N/A')}</p>
      <a href="{map_search_url}" target="_blank" class="map-link">📍 Ver no Google Maps</a>
      <a href="{calendar_shortcut_url}" target="_blank" class="calendar-link">🗓️ Marcar no Google Calendar (Escolher Data/Hora)</a>
    </div>
    """
    return html_content

def display_introducao_html():
    """Gera e retorna o HTML para a introdução estilizada do Exploraê."""
    return """
    <div class="introduction-box">
        <h1>✨ Bem-vindo(a) ao <span class="highlight">Exploraê</span>! ✨</h1>
        <p>Cansado(a) de pensar "O que eu faço hoje?" 🤔</p>
        <p>O <span class="highlight">Exploraê</span> é seu agente pessoal para descobrir lugares e experiências incríveis e <span class="highlight">reais</span> perto de você!</p>
        <p>Com a ajuda da Inteligência Artificial e da Busca do Google, ele encontra o rolê perfeito baseado no seu perfil.</p>
        <p>Vamos começar a explorar?</p>
    </div>
    """

def formatar_config_perfil_html():
    """Gera o HTML estilizado para a mensagem de configuração de perfil inicial."""
    return """
    <div class="config-message">
        <h2>✨ Bem-vindo(a) ao Exploraê!</h2>
        <p>Vamos configurar seu perfil para começar a explorar!</p>
    </div>
    """


def get_titulo_explorador(exploracoes):
    """Determina o título do usuário com base nos rolês explorados."""
    if exploracoes == 0:
        return "Aventureiro(a) Iniciante"
    elif exploracoes == 1:
        return "Primeira Exploração Desbloqueada!" # Conquista após o 1o
    elif 2 <= exploracoes <= 5:
        return "Explorador(a) de Primeira Viagem"
    elif 6 <= exploracoes <= 15:
        return "Conquistador(a) Local"
    elif 16 <= exploracoes <= 30:
        return "Embaixador(a) da Cidade"
    elif exploracoes >= 31:
        return "Mestre das Explorações Lendário(a)!"
    else:
        return "Nível Desconhecido"

In [44]:
# @title 5. Função Salvar Perfil (Opcional)
# @markdown Define a função para salvar o perfil.

# Para usar, certifique-se que seu Google Drive está montado na Célula 2
# from google.colab import drive
# drive.mount('/content/drive') # Se não montou antes

def salvar_perfil(user_profile, path=PROFILE_PATH):
    """Salva o dicionário do perfil em um arquivo JSON."""
    try:
        with open(path, 'w') as f:
            json.dump(user_profile, f, indent=4) # indent para formatar bonito no JSON
        # print(f"\n💾 Perfil salvo com sucesso em {path}.") # Opcional: Imprimir confirmação
    except Exception as e:
        print(f"\n⚠️ Não foi possível salvar o perfil: {e}")
        print("Verifique se o Google Drive está montado corretamente e se você tem permissão de escrita.")



In [ ]:
# @title 4. Fluxo Principal de Interação (Com Inputs Formatados)
# @markdown Inicia o fluxo principal, gera sugestões, gerencia interação e exibe resultados estilizados.

def executar_fluxo_sugestoes(user_profile):
    """Função principal para gerar e interagir com as sugestões."""

    # --- Exibir Introdução Estilizada ---
    # Agora exibida APENAS se o perfil está completo E cliente/modelo pronto (no bloco inicial abaixo)
    # display(HTML(display_introducao_html())) # Removida daqui


    # --- Função interna para gerar e exibir sugestões ---
    def obter_e_exibir_sugestoes(perfil, tentar_novas=False):
        print("🗺️ Preparando suas sugestões de aventura...")
        # Verifica se o cliente IA e o MODEL_ID estão disponíveis antes de tentar gerar
        if client is None or MODEL_ID is None:
            print("❌ Cliente Gemini ou Modelo ID não disponível para gerar sugestões. Verifique a Célula 1.")
            return [] # Retorna lista vazia se o modelo não estiver pronto

        sugestoes_texto_raw = gerar_sugestoes_texto(perfil, tentar_novas=tentar_novas)

        sugestoes_atuais_parsed = [] # Inicializa lista vazia
        if sugestoes_texto_raw:
            sugestoes_atuais_parsed = parse_sugestoes(sugestoes_texto_raw)

            if sugestoes_atuais_parsed:
                sugestoes_html = formatar_sugestoes_html(sugestoes_atuais_parsed)
                display(HTML(sugestoes_html))
            else:
                print("❌ Não foi possível extrair sugestões válidas da resposta da IA.")
                # Mesmo sem sugestões válidas, exibe as instruções para que o usuário saiba o que fazer
                display(HTML(formatar_instrucoes_input_html()))

        else:
            print("❌ Não foi possível gerar sugestões.")
            # Mesmo sem sugestões, exibe as instruções para que o usuário saiba o que fazer
            display(HTML(formatar_instrucoes_input_html()))


        return sugestoes_atuais_parsed # Retorna as sugestões parseadas


    # --- Geração e Exibição das Sugestões Iniciais ---
    # Garante que user_profile tem campos essenciais antes de gerar
    # Verifica apenas campos que são input obrigatório
    campos_essenciais = ['nome', 'tipo_role', 'horarios_que_costuma_ter_libre', 'cidade_e_bairro']
    # Verifica se o cliente/modelo está pronto E se os campos essenciais do perfil estão preenchidos (exceto 'nome', que já é verificado antes)
    if client is not None and MODEL_ID is not None and user_profile.get('nome') and all(user_profile.get(k) for k in campos_essenciais if k != 'nome'):
         # EXIBE A INTRODUÇÃO APENAS QUANDO TUDO ESTÁ PRONTO PARA INICIAR O FLUXO PRINCIPAL
         display(HTML(display_introducao_html()))
         sugestoes_atuais_parsed = obter_e_exibir_sugestoes(user_profile)
    else:
         print("\nCliente Gemini ou Modelo ID não configurado (Verifique Célula 1) ou Perfil do usuário incompleto/não criado (Verifique a Célula 2).")
         print("O fluxo de sugestões não pode ser iniciado.")
         # Exibe a instrução de configuração de perfil e as instruções de input caso o perfil esteja incompleto/cliente não pronto
         # display(HTML(formatar_config_perfil_html())) # Já exibido na Célula 2 se o perfil estava vazio
         display(HTML(formatar_instrucoes_input_html())) # Exibe as instruções para permitir a edição 'e'
         sugestoes_atuais_parsed = [] # Garante que a lista está vazia se não puder gerar


    # --- Loop de Interação com o Usuário ---
    # Verifica se há sugestões disponíveis antes de iniciar o loop
    # Note: Agora o loop pode começar mesmo sem sugestões, para permitir 'n' ou 'e'.
    # A condição 'if sugestoes_atuais_parsed:' agora apenas controla se o dicionário de sugestões está disponível para escolha 1, 2, 3.

    # Cria um dicionário mapeando o número da opção para o dicionário da sugestão (pode estar vazio inicialmente)
    sugestoes_dict = {i + 1: sug for i, sug in enumerate(sugestoes_atuais_parsed)}


    while True:
        # A interface de input fica ABAIXO da saída HTML estilizada
        # Note: O input em si não pode ser estilizado diretamente por HTML no Colab
        # Mas as instruções ACIMA dele são HTML.
        # --- INPUT PRINCIPAL COM QUEBRA DE LINHA NO FINAL (\n) ---
        escolha = input("Sua escolha (1, 2, 3), 'n' para novas, ou 'e' para editar perfil:\n").lower().strip() # \n no final

        if escolha in ['1', '2', '3']:
            try:
                opcao_escolhida_num = int(escolha)
                # Verifica se há sugestões disponíveis E se a opção escolhida está na lista atual
                if sugestoes_atuais_parsed and opcao_escolhida_num in sugestoes_dict:
                    sugestao_final = sugestoes_dict[opcao_escolhida_num]
                    # print(f"\n🥳 Você escolheu: {sugestao_final.get('NOME', 'Opção desconhecida')}") # Removido para usar o card HTML


                    # --- Exibir detalhes da escolha com HTML estilizado ---
                    display(HTML(formatar_detalhes_html(sugestao_final)))

                    # Removido: Lógica de Agendamento

                    # --- Atualizar Gamificação e Histórico (Simplificados) ---
                    user_profile['rolês_explorados'] = user_profile.get('rolês_explorados', 0) + 1
                    print(f"\nTotal de explorações concluídas: {user_profile['rolês_explorados']}")

                    # Adiciona o rolê ao histórico (simplificado)
                    rolê_historico_simples = {
                        'NOME': sugestao_final.get('NOME', 'Lugar Desconhecido'),
                        'data_escolha': datetime.now().isoformat() # Adiciona data da escolha
                    }
                    user_profile['historico_rolês'].append(rolê_historico_simples)
                    # Exibe histórico de forma simples, apenas os nomes no print
                    historico_nomes = [item.get('NOME', 'Lugar Desconhecido') for item in user_profile.get('historico_rolês', [])][-5:] # Últimos 5
                    print(f"📚 Histórico Recente: {', '.join(historico_nomes) if historico_nomes else 'Vazio'}")


                    # Exibe o título atualizado
                    titulo_atual = get_titulo_explorador(user_profile.get('rolês_explorados', 0))
                    print(f"👑 Seu Título Atual: {titulo_atual}")


                    # --- Incentivo ao Feedback ---
                    print("\n✨ Que legal! Espero que curta este rolê!")
                    print("Considere visitar o local e, se puder, deixar uma avaliação no Google Maps para ajudar a comunidade.")

                    # --- Salvar Perfil (Opcional) ---
                    salvar_perfil(user_profile) # Salva o perfil após a escolha, gamificação e histórico


                    # --- Fim da Interação para esta sugestão ---
                    print("\n--- Fim da Interação para esta sugestão. ---")
                    # Perguntar se quer ver mais sugestões ou sair
                    # --- INPUT CONTINUAR/SAIR COM QUEBRA DE LINHA NO FINAL (\n) ---
                    mais_sugestoes = input("Deseja ver mais sugestões? (s/n):\n").lower().strip() # \n no final
                    if mais_sugestoes != 's':
                         break # Sai do loop principal
                    else:
                         # Se quiser mais, gera novas sugestões e continua o loop
                         sugestoes_atuais_parsed = obter_e_exibir_sugestoes(user_profile, tentar_novas=True)
                         if sugestoes_atuais_parsed:
                             # Atualiza o dicionário de mapeamento APENAS se gerou novas sugestões
                             sugestoes_dict = {i + 1: sug for i, sug in enumerate(sugestoes_atuais_parsed)}
                         else:
                             print("😓 Não há sugestões disponíveis para continuar.")
                             break # Sai se não conseguir gerar mais


                else: # Se escolheu 1, 2, ou 3, mas não havia sugestões ou a opção era inválida para as sugestões atuais
                     print("❌ Opção inválida. Por favor, escolha 1, 2 ou 3, ou digite 'n' ou 'e'.")
            except ValueError:
                 print("❌ Entrada inválida. Por favor, digite o número da opção (1, 2, 3), 'n' ou 'e'.")

        elif escolha == 'n':
            # --- Novas Sugestões ---
            sugestoes_atuais_parsed = obter_e_exibir_sugestoes(user_profile, tentar_novas=True)
            if sugestoes_atuais_parsed:
                sugestoes_dict = {i + 1: sug for i, sug in enumerate(sugestoes_atuais_parsed)} # Atualiza o dicionário de mapeamento
            else:
                print("😓 Não há sugestões disponíveis para continuar.")
                break # Sai se não conseguir gerar mais

        elif escolha == 'e':
            # --- Editar Perfil ---
            editar_perfil(user_profile)
            # Após editar, sugerimos gerar novas sugestões com base no novo perfil
            print("\nPerfil atualizado. Gerando novas sugestões com base nas mudanças...")
            sugestoes_atuais_parsed = obter_e_exibir_sugestoes(user_profile, tentar_novas=True) # Tenta novas com base no perfil novo
            if sugestoes_atuais_parsed:
                sugestoes_dict = {i + 1: sug for i, sug in enumerate(sugestoes_atuais_parsed)} # Atualiza o dicionário de mapeamento
            else:
                 print("😓 Não há sugestões disponíveis com o perfil atualizado.")
                 # Se não gerou novas, o loop continua com as sugestões antigas (se houver)
                 if not sugestoes_dict: # Se não tinha nem antes, nem depois
                     break # Sai se não há sugestões para interagir

            # Salva o perfil após a edição
            salvar_perfil(user_profile)


        else:
            print("❓ Entrada inválida. Por favor, escolha 1, 2, 3, 'n' ou 'e'.")

    # --- Fim do Loop Principal ---
    # Mensagem final estilizada
    final_message_html = f"""
    <div class="final-message">
        <h2>🎉 Até a Próxima Exploração, {user_profile.get('nome', 'Explorador(a)')}!</h2>
        <p>Seu título final: <span class="title-highlight">{get_titulo_explorador(user_profile.get('rolês_explorados', 0))}</span></p>
        <p>Você explorou <span class="title-highlight">{user_profile.get('rolês_explorados', 0)}</span> rolês nesta sessão.</p>
        <p>Seu perfil e progresso foram salvos.</p>
    </div>
    """
    display(HTML(final_message_html))


# Este bloco 'if/else' deve estar alinhado ao nível global, fora da função.
# Ele inicia o fluxo principal se as configurações e o perfil estiverem prontos.
# Remova qualquer outra versão deste bloco que possa estar dentro da função.
# INÍCIO DO BLOCO QUE INICIA O FLUXO PRINCIPAL (ALINHADO À ESQUERDA)
campos_essenciais_para_iniciar = ['nome', 'tipo_role', 'horarios_que_costuma_ter_libre', 'cidade_e_bairro']
if client is not None and MODEL_ID is not None and user_profile.get('nome') and all(user_profile.get(k) for k in campos_essenciais_para_iniciar if k != 'nome'):
    executar_fluxo_sugestoes(user_profile)
else:
    print("\nCliente Gemini ou Modelo ID não configurado (Verifique Célula 1) ou Perfil do usuário incompleto/não criado (Verifique a Célula 2).")
    print("O fluxo de sugestões não pode ser iniciado.")
    # Exibe a instrução de configuração de perfil e as instruções de input caso o perfil esteja incompleto/cliente não pronto
    # display(HTML(formatar_config_perfil_html())) # Já exibido na Célula 2 se o perfil estava vazio
    display(HTML(formatar_instrucoes_input_html())) # Exibe as instruções para permitir a edição 'e'
# FIM DO BLOCO QUE INICIA O FLUXO PRINCIPAL

🗺️ Preparando suas sugestões de aventura...


Sua escolha (1, 2, 3), 'n' para novas, ou 'e' para editar perfil:
n
🗺️ Preparando suas sugestões de aventura...


Sua escolha (1, 2, 3), 'n' para novas, ou 'e' para editar perfil:
1



Total de explorações concluídas: 1
📚 Histórico Recente: Festa Vegana no Mundo Pensante
👑 Seu Título Atual: Primeira Exploração Desbloqueada!

✨ Que legal! Espero que curta este rolê!
Considere visitar o local e, se puder, deixar uma avaliação no Google Maps para ajudar a comunidade.

--- Fim da Interação para esta sugestão. ---
Deseja ver mais sugestões? (s/n):
s
🗺️ Preparando suas sugestões de aventura...


Sua escolha (1, 2, 3), 'n' para novas, ou 'e' para editar perfil:
e

--- Editar Seu Perfil ---
Deixe em branco para manter o valor atual.

Tipo de rolê atual (Quero fazer um aniversário): Novo tipo de rolê: 
fazer uma trilha
✅ Tipo de rolê atualizado.

Info complementar atual (comida vegana e bebidas): Nova info complementar: 
trilhas curtas
✅ Informação complementar atualizada.

Horários livres atuais (Final de semana): Novos horários livres: 
final de semana
✅ Horários livres atualizados.

Cidade e bairro atuais (São Paulo): Nova cidade e bairro: 
Curitiba, centro
✅ Cidade e bairro atualizada.
--- Edição Concluída ---

Perfil atualizado. Gerando novas sugestões com base nas mudanças...
🗺️ Preparando suas sugestões de aventura...
